In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import numpy as np
import torch
from pathlib import Path
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn as nn
import torch.nn.functional as F
from torchvision import models
import random
from collections import Counter
import cv2
import matplotlib.pyplot as plt 

In [2]:
PATH = Path('/home/ec2-user/whales2/imgs/')
folder = Path('/home/ec2-user/whales2/')

In [3]:
df = pd.read_csv('/home/ec2-user/whales2/train.csv')

In [4]:
train_names = df['Image'].values

In [5]:
# let's look at the typical size of these images
files = list((PATH).iterdir())
all_names = [f.parts[-1] for f in files]

In [6]:
test_names = list(set(all_names) - set(train_names))

In [8]:
len(all_names)

11468

In [10]:
len(all_names) - len(test_names)

4543

In [9]:
len(test_names)

6925

In [14]:
len(Counter(df['whaleID']))

447

In [13]:
Counter(df['whaleID'])

Counter({'whale_48813': 22,
         'whale_09913': 4,
         'whale_45062': 11,
         'whale_74162': 9,
         'whale_99558': 15,
         'whale_59255': 17,
         'whale_87291': 22,
         'whale_33152': 15,
         'whale_88147': 10,
         'whale_77693': 19,
         'whale_74625': 9,
         'whale_06069': 15,
         'whale_62939': 5,
         'whale_67036': 18,
         'whale_78785': 12,
         'whale_75682': 3,
         'whale_08017': 24,
         'whale_23574': 9,
         'whale_46169': 12,
         'whale_11242': 20,
         'whale_03227': 21,
         'whale_24815': 18,
         'whale_36231': 15,
         'whale_89615': 24,
         'whale_64989': 14,
         'whale_52342': 19,
         'whale_63948': 12,
         'whale_80947': 12,
         'whale_80247': 17,
         'whale_11076': 20,
         'whale_70138': 22,
         'whale_07863': 9,
         'whale_36300': 9,
         'whale_64634': 19,
         'whale_22448': 14,
         'whale_68116': 23,


In [7]:
def read_image(path):
    im = cv2.imread(str(path))
    return cv2.cvtColor(im, cv2.COLOR_BGR2RGB)

In [ ]:
dims = []
for f in files:
    try:
        dim = read_image(str(f)).shape
        dims.append(dim)
    except:
        print(f)

In [ ]:
# let's look at the typical size of these images
counts = Counter(dims)
counts

In [ ]:
proportions = [(k[1]/k[0], counts[k]) for k in counts]

In [ ]:
sorted(proportions, key=lambda x:x[1], reverse=True)

# Resizing Images

In [15]:
train_path = folder/"train-300"
valid_path = folder/"valid-300"
test_path = folder/"test-300"

train_path.mkdir(exist_ok=True)
valid_path.mkdir(exist_ok=True) 
test_path.mkdir(exist_ok=True)

In [16]:
def resize_images(path, sz=300):
    im = read_image(path)
    sz2 = int(1.75*sz)
    return cv2.resize(im, (sz2, sz))

In [17]:
def resize_all_images(train_path, valid_path,test_path, org_path, sz=300):
    np.random.seed(3)
    files = list((org_path).iterdir())
    for f in files:
        f_name = f.parts[-1]
        if f_name in test_names:
            new_path = test_path/f_name
        elif np.random.uniform() < 0.8:
            new_path = train_path/f_name
        else:
            new_path = valid_path/f_name
        im = resize_images(f)
        cv2.imwrite(str(new_path), cv2.cvtColor(im, cv2.COLOR_RGB2BGR))

In [19]:
#run just the first time
resize_all_images(train_path, valid_path,test_path, PATH)